In [3]:
import pandas as pd
import numpy as np

star_df = pd.read_csv('celeb_name_list.csv')        # 연예인 리스트
article_file = open('/Users/hayeong/DataScience/Data-Science/news_list.txt', 'r')      # 크롤링한 기사 제목들의 텍스트 파일 (구분자는 '\n'으로 가정)

# articles_dict: {name: <연예인 이름 리스트>, article: <기사 제목 리스트>}
articles_dict = {'Name':[], 'Article':[]}

star_df.head(10)

,ID,Name,Category,Age
0,2470,BMK,가수,47
1,2469,JK김동욱,가수,44
2,2468,KAI,가수,26
3,2467,MC딩동,배우,41
4,2466,MC몽,가수,41
5,2465,RM,가수,26
6,2464,T.O.P,가수,32
7,2463,가인,가수,33
8,2462,가희,배우,39
9,2461,간미연,가수,38


# articles_dict 생성 후 데이터프레임 만들기
: 기사 제목에 특정 연예인 이름이 들어있으면 'Article', 'Name' 키에 각각 저장 후 article_df 생성

* article: 기사 제목
* star_df: celeb_name_list.csv의 데이터프레임. star_df['Name']에 연예인 이름(=name)이 저장되어있음
* article_df: articles_dict의 데이터프레임. columns=['Name', 'Article'] 


In [5]:
while True:
    # 크롤링한 기사 제목 한 줄을 읽어옴
    article = article_file.readline()
    # 크롤링한 기사 제목을 모두 읽었으면 break
    if article == '':
        break
    #article = article[]       # 기사 제목 뒤의 신문사 이름 제외시키기
    # namelist 안의 모든 연예인 이름에 대해서 기사 제목에 포함되는 이름이 있는 지 확인
    for name in star_df['Name']:
        # 보통 연예인 이름 뒤에 컴마가 오길래 '<이름>,'를 찾는걸로 해봄
        if name+',' in article:
            if not (name in articles_dict['Name']):
                articles_dict['Name'].append(name)
                articles_dict['Article'].append(article+'\n')
            else:
                index = articles_dict['Name'].index(name)
                if not (article in articles_dict['Article'][index]):
                    articles_dict['Article'][index] += article+'\n'
                
        # 제목에 포함된 연예인을 찾았어도 연예인 리스트 전체 탐색함
        # => 기사에 두 명 이상의 연예인이 있을 경우 모두 찾음
article_df = pd.DataFrame(articles_dict)

In [6]:
article_df.head(10)

,Name,Article
0,김혜수,"김혜수, 건강미 넘치는 일상 '시선강탈'\n\n"
1,김성령,"김성령, 건강미 넘치는 프리다이빙…""54세 몸매 맞아?""\n\n"
2,이하늬,"이하늬, 크롭트 톱 입고 복근 공개…숨길 수 없는 건강미\n\n“클래스가 다른 아름..."
3,김희정,"김희정, 시선 강탈 구릿빛 섹시..건강미에 감탄\n\n김희정, 건강미 넘치는 화보 ..."
4,지우,"'48.6kg' 김지우, ♥레이먼킴도 놀랄 건강미 가득 보디라인\n\n'레이먼킴♥'..."
5,여진,"‘건강미 대명사’ 최여진, 보트 위 구릿빛 여신 [SNS★컷]\n\n최여진, 서핑보..."
6,진,"‘건강미 대명사’ 최여진, 보트 위 구릿빛 여신 [SNS★컷]\n\n‘친절한 진료실..."
7,최여진,"‘건강미 대명사’ 최여진, 보트 위 구릿빛 여신 [SNS★컷]\n\n최여진, 서핑보..."
8,류화영,"류화영, 건강미 돋보이는 테니스장에서의 하루\n\n"
9,신현빈,"신현빈, 건강미X고혹미 다 가진 매력부자 '심쿵주의보' [화보]\n\n"


# count_dict 생성 후 데이터프레임 만들기
: 각각의 연예인 이름이 몇 번 언급되어있는지를 저장함
'Name'키에 연예인 이름, 'Count'키에 언급 횟수를 저장 후 count_df 생성


In [7]:
# count_dict: {Name : 한 번 이상 언급된 연예인 이름 리스트, Count : 언급 횟수}
count_dict = {'Name': articles_dict['Name'], 'Count': []}
i = 0
for name in articles_dict['Name']:
    count_dict['Count'].append(articles_dict['Article'][i].count('\n'))
    i +=1

count_df = pd.DataFrame(count_dict)
count_df.head()

,Name,Count
0,김혜수,2
1,김성령,2
2,이하늬,26
3,김희정,34
4,지우,6


# 데이터프레임 전처리

1. 연예인 리스트 데이터프레임(star_df)과 연예인별 언급횟수 데이터프레임(count_df) 합침
2. 언급 되지 않은 연예인이 있는 행은 삭제 (count 컬럼이 NaN인 경우 해당 행 삭제)
3. count 컬럼을 기준으로 내림차순 정렬

In [8]:
# df에 언급 횟수인 Count 컬럼 추가
merged_df = pd.merge(star_df, count_df, on='Name', how = 'outer')
merged_df.head()

,ID,Name,Category,Age,Count
0,2470,BMK,가수,47,NaN
1,2469,JK김동욱,가수,44,NaN
2,2468,KAI,가수,26,NaN
3,2467,MC딩동,배우,41,NaN
4,2466,MC몽,가수,41,NaN


In [9]:
# 언급 되지 않은 연예인이 있는 행은 삭제
temp_df = merged_df.dropna(axis = 0, how = 'any')
# 언급 횟수 기준 내림차순으로 정렬
sorted_df = temp_df.sort_values(by = 'Count', ascending=False,)

# final_df columns: 연예인 정보 + 언급 횟수 + 언급된 기사 제목
final_df = pd.merge(sorted_df, article_df, on='Name', how = 'outer')
final_df.head(20)

,ID,Name,Category,Age,Count,Article
0,1127,유이,배우,32,62.0,"유이, 피트니스 콘텐츠 구독자 수 10만 돌파..건강미 넘치는 행보\n\n‘나 혼자..."
1,362,진,가수,27,46.0,"‘건강미 대명사’ 최여진, 보트 위 구릿빛 여신 [SNS★컷]\n\n‘친절한 진료실..."
2,882,이시영,배우,38,42.0,"이시영, 흑백 뚫고 나오는 걸크러시…만능 운동러의 건강미\n\n이시영, 건강미 넘치..."
3,1477,설현,가수,25,38.0,"AOA 설현, 농심 '켈로그' 모델 발탁…""건강미 대표 이미지 부합""\n\nAOA ..."
4,1923,김희정,배우,49,34.0,"김희정, 시선 강탈 구릿빛 섹시..건강미에 감탄\n\n김희정, 건강미 넘치는 화보 ..."
5,1922,김희정,배우,28,34.0,"김희정, 시선 강탈 구릿빛 섹시..건강미에 감탄\n\n김희정, 건강미 넘치는 화보 ..."
6,1460,소유,가수,28,30.0,"소유, 신곡 '가라고' 안무 영상 공개…탄탄한 건강미\n\n소유, 나이 29살 싱글..."
7,746,이하늬,배우,37,26.0,"이하늬, 크롭트 톱 입고 복근 공개…숨길 수 없는 건강미\n\n“클래스가 다른 아름..."
8,1487,선미,가수,28,24.0,"‘8kg 증량’ 선미, 50kg 인증…건강미 UP\n\n8Kg 늘린 선미, 건강미 ..."
9,6,효린,가수,29,20.0,"효린, 'SAY MY NAME' 첫 뮤비티저 공개 예고…'건강미 폭발' 퍼포먼스 기..."


# 데이터 확인하기

1. 동명이인인 데이터 확인
2. 상위 20명의 후보 연예인 리스트 확인
3. 후보 연예인들이 언급되었던 기사가 제대로 된 것인지 확인
4. 최종 데이터프레임을 csv 파일로 저장

In [10]:
### 생략 가능 ###
# 동명이인일 시 두 명 다 같은 언급횟수를 가지고 있음  
# ex)김희정(49세), 김희정(28세): 기사 하나에 언급되면 둘 다 count = 1이 됨
# duplicated: 동명이인인 이름의 리스트
duplicated = final_df[final_df.Name.duplicated()]['Name'][:20].tolist()
print('=== 동명이인인 이름 리스트 ===')
print(duplicated)


# 동명이인이 들어간 기사 체크하기
for key in duplicated:
    print('>> '+key+'\n' + articles_dict['Article'][articles_dict['Name'].index(key)])

# 동명이인의 정보 확인하기
duplicated_df = final_df[final_df['Name'].isin(duplicated)]
duplicated_df.head()

=== 동명이인인 이름 리스트 ===
['김희정', '김종국', '현아', '한혜진', '지성', '김지훈', '김지훈', '미나', '은정']
>> 김희정
김희정, 시선 강탈 구릿빛 섹시..건강미에 감탄

김희정, 건강미 넘치는 화보 ‘치명적인 매력’

김희정, 군살 제로 건강미..튼튼 꿀벅지 '깜짝'[★SHOT!]

김희정, 독보적 건강미 발산…다이어트도 힙하게

김희정, 다이어트 CF로 건강미 과시

김희정, 구릿빛 피부에 빛나는 몸매…건강미 넘치네

김희정, 건강미에 더한 몽환적 분위기…치명적인 매력 [화보]

김희정, 화보 비하인드컷 공개...'러블리+건강미'

김희정, 건강미 넘치는 '워너비 몸매' 담은 티저 영상

"건강미가, 물씬"…김희정, 화보 비하인드컷도 독보적

김희정, '건강미+몽환적 분위기' 발산하는 유니크한 매력 [화보]

김희정, 구릿빛 피부+탄탄 몸매로 '섹시 건강미' 뿜뿜

김희정, 비키니·트레이닝복까지 완벽 소화…건강미 넘치는 매력 [화보]

김희정, 준PO 4차전 시구..복근 드러낸 '건강미'

배우 김희정, SNS에 건강미 넘치는 수영복 차림 공개

김희정, '힙'한 다이어트 티저 영상 공개…건강미 넘치는 '워너비 몸매'

김희정, 건강미 넘치는 구릿빛 피부


>> 김종국
[포토] 김종국, 한결같은 건강미

[포토] 김종국, 넘치는 건강미

[포토] 김종국, '넘치는 건강미'

김종국, 흘러넘치는 건강미 '퍼펙트 하죠?' [MD동영상]

[포토] 김종국, 건강미 가득

[포토]김종국, 건강미 넘치는 브이

[포토] 김종국, 건강미 철철~

[포토]김종국, 건강미 넘쳐


>> 현아
‘원더풀’ 현아, 美친 건강미 [화보]

현아, 화보에서 美친 건강미 과시

현아, 건강미 넘치는 스포티+섹시美 [화보]

현아, 美친 보디라인 자랑…넘사벽 건강미에 감탄이 절로 [화보]

현아, 속옷 화보서 탄탄 바디라인 과시…'건강미 폭발'

현아, 스포티한 레깅스 패션…건강미 '물씬'

[포토] 현아, 건강미 발산


>> 한혜진
[포토] 

,ID,Name,Category,Age,Count,Article
4,1923,김희정,배우,49,34.0,"김희정, 시선 강탈 구릿빛 섹시..건강미에 감탄\n\n김희정, 건강미 넘치는 화보 ..."
5,1922,김희정,배우,28,34.0,"김희정, 시선 강탈 구릿빛 섹시..건강미에 감탄\n\n김희정, 건강미 넘치는 화보 ..."
13,2038,김종국,개그맨,57,16.0,"[포토] 김종국, 한결같은 건강미\n\n[포토] 김종국, 넘치는 건강미\n\n[포토..."
14,2039,김종국,가수,44,16.0,"[포토] 김종국, 한결같은 건강미\n\n[포토] 김종국, 넘치는 건강미\n\n[포토..."
17,75,현아,가수,33,14.0,"‘원더풀’ 현아, 美친 건강미 [화보]\n\n현아, 화보에서 美친 건강미 과시\n\..."


In [11]:
candidates = final_df['Name'].tolist()[:20]
print('=== Top 20 ===')
print(candidates)

=== Top 20 ===
['유이', '진', '이시영', '설현', '김희정', '김희정', '소유', '이하늬', '선미', '효린', '수현', '클라라', '조여정', '김종국', '김종국', '가희', '엘', '현아', '현아', '예진']


In [12]:
# 상위 20명의 기사가 제대로 된 것인지 체크하기
print('=== Check articles ===')
for key in candidates:
    print('>> '+key+'\n' + articles_dict['Article'][articles_dict['Name'].index(key)])

=== Check articles ===
>> 유이
유이, 피트니스 콘텐츠 구독자 수 10만 돌파..건강미 넘치는 행보

‘나 혼자 산다’ 유이, 건강미 폭발하는 폴댄스 공개

유이, 티셔츠+레깅스 조합이 이렇게 잘 어울리다니…‘건강미’ 넘치는 근황

유이, 탄탄한 바디라인 드러내는 건강미 넘치는 화보 공개

[오마이포토] 'SF8' 유이, 건강미 넘치는 미소

유이, 운동으로 완성된 완벽 S라인 '건강미 발산'

[화보IS] 유이, 운동으로 완성한 탄탄한 건강미

[화보] 유이, 파격 세미 누드로 뽐낸 건강미

배우 유이, 건강미 넘치는 행보 눈길

“건강미란 이런 것”...유이, 하의실종 파격 화보 공개 [EN스타]

유이, 파격 세미누드 화보 공개..'꿀벅지' 건강미 되찾았다 [화보]

유이, 아찔+과감한 화보 통해 폴댄스로 다져진 건강미 발산

유이, 아름다운 보디라인+건강미...세미누드 화보 공개

유이, 완벽하게 되찾은 건강미..인형 같은 제주 소녀 [화보]

'나혼산' 유이, 건강미 폭발 파워 폴댄스→숨겨온 솔직한 속내까지 [Oh!쎈 예...

유이, 몸매 이정도였어? 탄탄한 건강미

유이, 되찾은 건강미…리즈는 지금 [화보]

유이, 핫보디 아이콘의 아름다운 건강미(화보)

유이, 숲속에서 우아한 화보 ‘건강미 넘치네’

유이, 바다 앞 여유로운 미소 ‘건강미’ 뿜뿜 [SNS★컷]

유이, 거식증 루머 심경고백→'파격' 세미누드 공개→건강美 발산[종합]

유이, 과감한 세미 누드…아름다운 건강美 [화보]

'나 혼자 산다' 유이, 폴댄스로 탄탄한 몸매+건강미 과시

유이, 요가로 다져진 건강미…"나를 사랑해주는 것이 건강한 삶"

유이, 꿀벅지 레전드 다운 핫 바디 공개...건강미 넘치는 싱글즈 7월 호 커버

［화보］유이, 건강미 넘치는 탄탄한 바디라인+독보적 비주얼 ‘시선강탈’

'나 혼자 산다' 유이, 건강미 폭발하는 풀 파워 폴 댄스! 파워는 완성! 유연성...

유이, 깡마른 몸매에도 살아있는 '건강미'[★SHOT!]

[POP이슈]유이

In [ ]:
import qgrid
qgrid_widget = qgrid.show_grid(final_df,show_toolbar=True)
qgrid_widget

In [13]:
# csv로 저장
final_df.to_csv('candidates.csv', index=False, mode='w', encoding='utf-8-sig')
final_df.head(20)

,ID,Name,Category,Age,Count,Article
0,1127,유이,배우,32,62.0,"유이, 피트니스 콘텐츠 구독자 수 10만 돌파..건강미 넘치는 행보\n\n‘나 혼자..."
1,362,진,가수,27,46.0,"‘건강미 대명사’ 최여진, 보트 위 구릿빛 여신 [SNS★컷]\n\n‘친절한 진료실..."
2,882,이시영,배우,38,42.0,"이시영, 흑백 뚫고 나오는 걸크러시…만능 운동러의 건강미\n\n이시영, 건강미 넘치..."
3,1477,설현,가수,25,38.0,"AOA 설현, 농심 '켈로그' 모델 발탁…""건강미 대표 이미지 부합""\n\nAOA ..."
4,1923,김희정,배우,49,34.0,"김희정, 시선 강탈 구릿빛 섹시..건강미에 감탄\n\n김희정, 건강미 넘치는 화보 ..."
5,1922,김희정,배우,28,34.0,"김희정, 시선 강탈 구릿빛 섹시..건강미에 감탄\n\n김희정, 건강미 넘치는 화보 ..."
6,1460,소유,가수,28,30.0,"소유, 신곡 '가라고' 안무 영상 공개…탄탄한 건강미\n\n소유, 나이 29살 싱글..."
7,746,이하늬,배우,37,26.0,"이하늬, 크롭트 톱 입고 복근 공개…숨길 수 없는 건강미\n\n“클래스가 다른 아름..."
8,1487,선미,가수,28,24.0,"‘8kg 증량’ 선미, 50kg 인증…건강미 UP\n\n8Kg 늘린 선미, 건강미 ..."
9,6,효린,가수,29,20.0,"효린, 'SAY MY NAME' 첫 뮤비티저 공개 예고…'건강미 폭발' 퍼포먼스 기..."
